In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [5]:
scales = [2]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.pvariance(x)

error = 0.01

# Prover/ data owner side
_, prover_model = computation_to_model(computation, precal_witness_path, True, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inf

==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":1,"lookup_range":[-4512,4800],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":9957,"total_const_size":3628,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"required_range_checks":[[-2,2]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715154745619}


In [7]:
# Prover/ data owner side
_, verifier_model = computation_to_model(computation, precal_witness_path, False,error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.7300560474395752 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 211.75
==== Generating Proof ====
proof:  {'instances': [['31220e5513f6e9b7d0f59d8f019cd197642a431b2136f0311abf6046c9006227', '0100000000000000000000000000000000000000000000000000000000000000', '4f03000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x20d9b05e0301c1f58990fbbcfd4d15541063bb7e5f53debb62129f9f86740ad21d8eba12250466fcc38266ad9fe608f6afe9f524558bbf7f2392fdb0b0325b6f1a58b64f26ff66800a459a3a1c4d056229aede9fe8ed3930de8e4f07787579101867fa5e7d9747611ca4200b98021cd5b747c7b322148c1ee3556d143e0dc91d27ae0caf4ad61a49f76c5aed8071470c24909eba4c9da1462be65afd8bc8d3b40c7754b3a7dbc3b9dfc06ed2c78367002526e515ed0d519d6d3c70ef1601e7530fe65adc8a402748397969ff8e59bdfa95dc18826dd0d55437957f7e6b6e05bf0288c472147a4a4b177975abf1e0c2d8dea3f2667e6861b5d3e9e1e911a295f2065a5b0c6cbd16cbff192dd36eb82cda93d4213675d7b4ede8d0a5e5f1bf8a1e17684e548009d

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [211.75]
